### Clean Training Data
#### Inputs
- Flow data from Sensirion data gathering software (csv)
- Sensor data from Read_Write_Sensor python script

#### Outputs
- Export Flow csv file
- Export SensorData csv file

#### Notes
- Make sure to line up timescales prior to exporting
- Next step is to scale in Matlab

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import time
import datetime

df = pd.read_excel('Training_2.xlsx')
flow = pd.read_csv('Training_2_Flow.csv')

In [2]:
minutes = []

for i in range(0, len(flow)):
    minutes.append(float(flow['Time'][i].split(':')[1]))
    
flow['Min'] = minutes
seconds = []

for i in range(0, len(flow)):
    seconds.append(float(flow['Time'][i].split(':')[2]) + (float(flow['Min'][i]) * 60))
    
flow['Seconds'] = seconds
flow.drop('Date', inplace = True, axis = 1)
flow.drop('Sample #', inplace = True, axis = 1)
flow.drop('Time', inplace = True, axis = 1)
flow.drop('Min', inplace = True, axis = 1)
flow.columns = ['Flow', 'Time']
flow.head()

,Flow,Time
0,0.033,2159.1057
1,0.033,2159.1062
2,0.033,2159.1067
3,0.000,2159.1072
4,0.033,2159.1077


In [3]:
df.head()

,Unnamed: 0,Sensor 1,Sensor 2,Sensor 3,Sensor 4,d_Sensor 1,d_Sensor 2,d_Sensor 3,d_Sensor 4,timeTracker
0,0,0.425000,0.435000,0.405000,0.425,0.000000,0.000000,0.000000,0.0,1.572986e+09
1,1,0.425000,0.430000,0.405000,0.425,0.000000,-0.244389,0.000000,0.0,1.572986e+09
2,2,0.418333,0.414900,0.395000,0.425,-0.205936,-0.750222,-0.308904,0.0,1.572986e+09
3,3,0.414950,0.414926,0.397556,0.425,-0.253372,-0.370822,-0.079622,0.0,1.572986e+09
4,4,0.408825,0.417003,0.399083,0.425,-0.370188,-0.084683,0.017312,0.0,1.572986e+09


In [4]:
timeSeconds = []

for i in range(0, len(df)):
    t = datetime.datetime.fromtimestamp(df['timeTracker'][i]).strftime('%Y-%m-%d %H:%M:%S.%f')
    seconds = float(t.split(':')[2])
    minutes = float(t.split(':')[1])
    timeSeconds.append(seconds + (minutes * 60))
    
df.drop('timeTracker', inplace = True, axis = 1)
df['Time'] = timeSeconds
df.drop('Unnamed: 0', inplace = True, axis = 1)
df.columns = ['1', '2', '3', '4', 'd1', 'd2', 'd3', 'd4', 'time']

In [6]:
df.head(100)

,1,2,3,4,d1,d2,d3,d4,time
0,0.425000,0.435000,0.405000,0.425000,0.000000,0.000000,0.000000,0.000000,2161.631985
1,0.425000,0.430000,0.405000,0.425000,0.000000,-0.244389,0.000000,0.000000,2161.642214
2,0.418333,0.414900,0.395000,0.425000,-0.205936,-0.750222,-0.308904,0.000000,2161.653005
3,0.414950,0.414926,0.397556,0.425000,-0.253372,-0.370822,-0.079622,0.000000,2161.663050
4,0.408825,0.417003,0.399083,0.425000,-0.370188,-0.084683,0.017312,0.000000,2161.672467
...,...,...,...,...,...,...,...,...,...
95,0.474114,0.490074,0.515832,0.426937,0.005924,0.014992,0.097372,0.011369,2162.591135
96,0.473722,0.489740,0.516098,0.427112,0.003626,0.012469,0.093227,0.011578,2162.601887
97,0.471907,0.489415,0.514643,0.427280,-0.005488,0.010175,0.080060,0.011735,2162.612838
98,0.471539,0.488881,0.514653,0.427443,-0.007225,0.006943,0.074660,0.011871,2162.623587


In [7]:
flow[flow['Time'] > df['time'][0]]

,Flow,Time
4945,0.000,2161.6326
4946,0.000,2161.6332
4947,-0.033,2161.6338
4948,-0.033,2161.6344
4949,0.000,2161.6350
...,...,...
242311,0.000,2282.6243
242312,0.000,2282.6248
242313,0.000,2282.6253
242314,0.000,2282.6258


In [8]:
flow[flow['Time'] < df['time'][len(df)-1]]

,Flow,Time
0,0.033,2159.1057
1,0.033,2159.1062
2,0.033,2159.1067
3,0.000,2159.1072
4,0.033,2159.1077
...,...,...
214008,22.300,2268.1853
214009,22.367,2268.1859
214010,22.400,2268.1865
214011,22.400,2268.1871


In [9]:
flow = flow[4945:214012]

In [10]:
export_csv = flow.to_csv(r'/Users/mikefurr/Documents/GitHub/coral_host/Jupyter Notebooks/flow.csv', header = True, index = None)
export_csv = df.to_csv(r'/Users/mikefurr/Documents/GitHub/coral_host/Jupyter Notebooks/sensors.csv', header = True, index = None)